# Contract Modification Risk Analysis for Federal Security Vendors (CORRECTED)

**⚠️ Important Note**: This analysis has been corrected to eliminate look-ahead bias. Previous versions incorrectly used `current_total_value_of_award` and `potential_total_value_of_award` which contain information from future modifications. The current version uses only data available at the time of the base award (modification_number = 0).

**Features used** (no look-ahead bias):
- `base_and_exercised_options_value` (initial base value)
- `base_and_all_options_value` (initial ceiling/maximum agreed)
- `federal_action_obligation` (initial funding obligation)
- Derived features: `options_vs_base_delta` (ceiling - base), duration, competition metrics

This ensures the model can only use information that would have been available to decision-makers at the time of contract award.


# Contract Modification Risk Analysis for Federal Security Vendors

This notebook executes the "Contract Modification Risk Analysis" scenario from `docs/strategic_analyses_security_companies.md` using the filtered USAspending prime transactions described in `docs/prime_transactions.md`. All commentary is provided in formal business English to support executives evaluating NAICS 561612 opportunities.



## Analytical framing

The investigation pursues three complementary objectives:

1. Quantify how frequently base awards evolve through downstream modifications across the security services market.
2. Surface agency-level and contract-structure patterns that increase modification risk, with emphasis on solicitation and pricing signals cited in the strategy memo.
3. Train and interpret a predictive model (80/20 train-test split) that ranks upcoming opportunities by modification risk, enabling proactive account planning.


In [28]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from IPython.display import display

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import roc_curve


def locate_repo_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / 'AGENTS.md').is_file() and (candidate / 'scripts').is_dir():
            return candidate
    raise RuntimeError(f'Unable to locate repository root from {start}')

PROJECT_ROOT = locate_repo_root(Path.cwd().resolve())

# Ensure PROJECT_ROOT is first in path to override any installed 'scripts' package
if str(PROJECT_ROOT) in sys.path:
    sys.path.remove(str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT))

# Restart the import system for 'scripts' module
modules_to_delete = [mod for mod in sys.modules if mod.startswith('scripts')]
for mod in modules_to_delete:
    del sys.modules[mod]

from scripts.contract_modification_risk import (
    build_contract_modification_dataset,
    train_modification_risk_classifier,
    NUMERIC_FEATURES,
    CATEGORICAL_FEATURES,
    BOOLEAN_FEATURES,
    TARGET_COLUMN,
)

pd.set_option('display.max_columns', 40)
pd.options.display.float_format = '{:,.2f}'.format
init_notebook_mode(connected=True)

RANDOM_STATE = 42



## Load the curated NAICS 561612 data set

The helper in `scripts/contract_modification_risk.py` ingests the filtered SQLite extract, keeps only base awards (`modification_number = '0'`), and attaches a boolean target that flags whether subsequent modifications exist for the same contract award key.


In [ ]:
dataset = build_contract_modification_dataset()
feature_columns = list(NUMERIC_FEATURES) + list(CATEGORICAL_FEATURES) + list(BOOLEAN_FEATURES)

# Convert boolean features to int to ensure compatibility with pipeline
for bool_col in BOOLEAN_FEATURES:
    if bool_col in dataset.columns:
        dataset[bool_col] = dataset[bool_col].astype(int)

print(f"Contracts analyzed: {len(dataset):,}")
print(f"Fields leveraged for modeling: {len(feature_columns)}")
print(f"  - Numeric: {len(NUMERIC_FEATURES)}")
print(f"  - Categorical: {len(CATEGORICAL_FEATURES)}")
print(f"  - Boolean: {len(BOOLEAN_FEATURES)}")

preview_columns = [
    'contract_award_unique_key',
    'awarding_agency_name',
    'type_of_contract_pricing',
    'solicitation_procedures',
    'base_and_all_options_value',
    'planned_duration_days',
    'number_of_offers_received',
    TARGET_COLUMN,
]

display(dataset[preview_columns].head())

Contracts analyzed: 46,771
Fields leveraged for modeling: 168
  - Numeric: 14
  - Categorical: 68
  - Boolean: 86


,contract_award_unique_key,awarding_agency_name,type_of_contract_pricing,solicitation_procedures,base_and_all_options_value,planned_duration_days,number_of_offers_received,has_modification
4,CONT_AWD_05GA0A19K0024_0559_05GA0A18A0002_0559,Government Accountability Office,FIRM FIXED PRICE,SIMPLIFIED ACQUISITION,"10,941.60",25.00,1.00,False
10,CONT_IDV_OASCTIA1401_1100,Executive Office of the President,FIRM FIXED PRICE,SUBJECT TO MULTIPLE AWARD FAIR OPPORTUNITY,0.00,NaN,8.00,False
13,CONT_AWD_0001_1100_OASCTIA1401_1100,Executive Office of the President,FIRM FIXED PRICE,SUBJECT TO MULTIPLE AWARD FAIR OPPORTUNITY,"90,000.00",169.00,4.00,True
14,CONT_AWD_0001_1100_USTGENB9005_1100,Executive Office of the President,NOT REPORTED,ONLY ONE SOURCE,0.00,"1,451.00",0.00,True
15,CONT_AWD_0004_1100_USTAGBD1502_1100,Executive Office of the President,LABOR HOURS,ONLY ONE SOURCE,"175,295.52",14.00,1.00,True



## Baseline contract modification incidence

The strategic memo prioritizes modification risk because it distorts both delivery costs and upside potential. The table below summarizes the incidence rate and provides quick context on market coverage.


In [26]:

mod_rate = dataset[TARGET_COLUMN].mean()
base_value = dataset['base_and_all_options_value']

summary = pd.DataFrame(
    {
        'Contracts with downstream modifications (%)': [mod_rate * 100],
        'Total base awards analyzed': [len(dataset)],
        'Agencies represented': [dataset['awarding_agency_name'].nunique()],
        'Awarding offices represented': [dataset['awarding_office_name'].nunique()],
        'Median base value (USD)': [base_value.median()],
        'Average planned duration (days)': [dataset['planned_duration_days'].median()],
    }
)

display(summary)


,Contracts with downstream modifications (%),Total base awards analyzed,Agencies represented,Awarding offices represented,Median base value (USD),Average planned duration (days)
0,81.13,46771,45,1326,"25,000.00",364.00



## Agency-level view of modification risk

Agencies vary widely in their propensity to adjust security contracts. Understanding that dispersion helps prioritize capture strategies and escalation readiness.


In [30]:

top_agencies = (
    dataset
    .groupby('awarding_agency_name')
    .agg(
        awards=('contract_award_unique_key', 'nunique'),
        modification_rate=(TARGET_COLUMN, 'mean'),
        total_base_value=('base_and_all_options_value', 'sum'),
    )
    .sort_values('modification_rate', ascending=False)
    .head(12)
    .reset_index()
)
top_agencies['mod_rate_pct'] = top_agencies['modification_rate'] * 100
top_agencies['total_base_value_m'] = top_agencies['total_base_value'] / 1e6

display(top_agencies[['awarding_agency_name', 'awards', 'mod_rate_pct', 'total_base_value_m']])

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Bar(
        x=top_agencies['mod_rate_pct'],
        y=top_agencies['awarding_agency_name'],
        orientation='h',
        name='Contracts modified (%)',
        marker_color='#1f77b4',
        hovertemplate='%{y}<br>Modification rate: %{x:.1f}%<extra></extra>',
    ),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(
        x=top_agencies['total_base_value_m'],
        y=top_agencies['awarding_agency_name'],
        mode='lines+markers',
        name='Base value (USD millions)',
        marker=dict(color='#ff7f0e', size=8),
        hovertemplate='%{y}<br>Base value: %{x:.1f}M<extra></extra>',
    ),
    secondary_y=True,
)
fig.update_layout(
    title='Agencies with the highest modification intensity',
    template='plotly_white',
    height=520,
    legend=dict(orientation='h', x=0, y=1.1),
)
fig.update_yaxes(categoryorder='total ascending')
fig.update_xaxes(title='Modification rate (%)')
fig.update_yaxes(title='Agency', secondary_y=False)
fig.update_yaxes(title='Base value (USD millions)', secondary_y=True)
iplot(fig)


,awarding_agency_name,awards,mod_rate_pct,total_base_value_m
0,Consumer Financial Protection Bureau,15,100.00,56.46
1,Corporation for National and Community Service,9,100.00,4.87
2,Consumer Product Safety Commission,1,100.00,0.87
3,John F. Kennedy Center for the Performing Arts,3,100.00,48.76
4,Federal Communications Commission,10,100.00,4.94
5,Merit Systems Protection Board,6,100.00,0.28
6,National Transportation Safety Board,4,100.00,7.45
7,National Science Foundation,1,100.00,0.01
8,National Labor Relations Board,1,100.00,0.03
9,Nuclear Regulatory Commission,4,100.00,24.36



## Contract size, schedule, and modification pressure

Value and timeline commitments often drive scope changes. Segmenting the portfolio into value deciles reveals how risk escalates together with deal size and delivery horizon.


In [31]:

value_ready = dataset.dropna(subset=['base_and_all_options_value']).copy()
value_ready = value_ready[value_ready['base_and_all_options_value'] > 0]
value_ready['value_segment'] = pd.qcut(value_ready['base_and_all_options_value'], 10, duplicates='drop')

value_summary = (
    value_ready
    .groupby('value_segment')
    .agg(
        awards=('contract_award_unique_key', 'nunique'),
        modification_rate=(TARGET_COLUMN, 'mean'),
        median_duration=('planned_duration_days', 'median'),
    )
    .reset_index()
)

def segment_label(interval):
    if pd.isna(interval):
        return 'Unknown'
    left = interval.left / 1e6
    right = interval.right / 1e6
    return f"${left:,.1f}M – ${right:,.1f}M"

value_summary['segment_label'] = value_summary['value_segment'].apply(segment_label)
value_summary['mod_rate_pct'] = value_summary['modification_rate'] * 100

display(value_summary[['segment_label', 'awards', 'mod_rate_pct', 'median_duration']])

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Bar(
        x=value_summary['segment_label'],
        y=value_summary['mod_rate_pct'],
        name='Contracts modified (%)',
        marker_color='#17becf',
    ),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(
        x=value_summary['segment_label'],
        y=value_summary['median_duration'],
        name='Median planned duration (days)',
        mode='lines+markers',
        marker=dict(color='#7f7f7f'),
    ),
    secondary_y=True,
)
fig.update_layout(
    title='Modification exposure by contract value decile',
    template='plotly_white',
    height=540,
    xaxis_title='Base and all options value decile',
)
fig.update_yaxes(title='Modification rate (%)', secondary_y=False)
fig.update_yaxes(title='Median duration (days)', secondary_y=True)
iplot(fig)


/tmp/ipykernel_17928/2402216978.py:7: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,segment_label,awards,mod_rate_pct,median_duration
0,$0.0M – $0.0M,4555,75.57,362.00
1,$0.0M – $0.0M,4492,70.08,335.00
2,$0.0M – $0.0M,4751,70.74,364.00
3,$0.0M – $0.0M,4491,71.45,364.00
4,$0.0M – $0.0M,4329,92.15,364.00
5,$0.0M – $0.1M,4522,73.95,364.00
6,$0.1M – $0.1M,4563,85.82,364.00
7,$0.1M – $0.4M,4484,85.82,122.00
8,$0.4M – $2.1M,4523,93.15,221.00
9,"$2.1M – $3,849.5M",4524,93.30,364.00



## Predictive model with 80/20 split

A HistGradientBoostingClassifier pipeline fits the engineered features (value deltas, competition, solicitation structure) with an 80% training and 20% hold-out testing split. Scores are interpreted as the probability that a base award generates at least one modification.


In [40]:

artifacts = train_modification_risk_classifier(
    dataset,
    test_size=0.2,
    random_state=RANDOM_STATE,
)

metrics_df = (
    pd.Series(artifacts.metrics)
    .rename_axis('metric')
    .to_frame('value')
    .sort_values('value', ascending=False)
)

display(metrics_df)
print(f"Train observations: {len(artifacts.X_train):,}")
print(f"Test observations: {len(artifacts.X_test):,}")
print("\nClassification report (test set):")
print(artifacts.classification_report_text)
print("Confusion matrix (rows=true class, cols=predicted class):")
print(artifacts.confusion_matrix)


,value
metric,
average_precision,0.99
recall,0.97
roc_auc,0.95
f1,0.95
precision,0.93
accuracy,0.91


Train observations: 37,416
Test observations: 9,355

Classification report (test set):
              precision    recall  f1-score   support

           0      0.843     0.668     0.745      1766
           1      0.926     0.971     0.948      7589

    accuracy                          0.914      9355
   macro avg      0.885     0.819     0.847      9355
weighted avg      0.911     0.914     0.910      9355

Confusion matrix (rows=true class, cols=predicted class):
[[1179  587]
 [ 219 7370]]


## Model diagnostics for executive review

To communicate practical effectiveness we pair the scorecard with familiar diagnostics: ROC curve to show true-positive lift, precision-recall curve for imbalance resilience, and a confusion-matrix heatmap that translates hits/misses into tangible counts.

In [32]:

fpr, tpr, roc_thresholds = roc_curve(artifacts.y_test, artifacts.y_pred_proba)
precision, recall, pr_thresholds = artifacts.precision_recall_curve
cmatrix = artifacts.confusion_matrix

roc_fig = go.Figure()
roc_fig.add_trace(
    go.Scatter(
        x=fpr,
        y=tpr,
        mode='lines',
        name='Model ROC',
        line=dict(color='#1f77b4', width=3),
        hovertemplate='False-positive rate: %{x:.3f}<br>True-positive rate: %{y:.3f}<extra></extra>'
    )
)
roc_fig.add_trace(
    go.Scatter(
        x=[0, 1],
        y=[0, 1],
        mode='lines',
        name='Chance line',
        line=dict(color='#aaaaaa', dash='dash')
    )
)
roc_fig.update_layout(
    title='ROC curve — probability of catching future modifications',
    xaxis_title='False-positive rate',
    yaxis_title='True-positive rate',
    template='plotly_white',
    height=420
)
iplot(roc_fig)

pr_fig = go.Figure()
pr_fig.add_trace(
    go.Scatter(
        x=recall,
        y=precision,
        mode='lines',
        name='Precision-Recall',
        line=dict(color='#d62728', width=3),
        hovertemplate='Recall: %{x:.3f}<br>Precision: %{y:.3f}<extra></extra>'
    )
)
pr_fig.update_layout(
    title='Precision–recall curve — focus on reliable warnings',
    xaxis_title='Recall (share of modified awards captured)',
    yaxis_title='Precision (share of flagged awards that truly modify)',
    template='plotly_white',
    height=420,
    yaxis=dict(range=[0, 1.05])
)
iplot(pr_fig)

cm_text = [[f"{val:,}" for val in row] for row in cmatrix]
conf_fig = go.Figure(
    data=go.Heatmap(
        z=cmatrix,
        x=['Predicted no change', 'Predicted modification'],
        y=['Actual no change', 'Actual modification'],
        colorscale='Blues',
        showscale=False,
        text=cm_text,
        texttemplate='%{text}'
    )
)
conf_fig.update_layout(
    title='Confusion matrix — how the model performs on the hold-out test set',
    xaxis_title='Predicted class',
    yaxis_title='Actual class',
    template='plotly_white',
    height=420
)
iplot(conf_fig)


## Feature drivers of modification risk

Permutation importance clarifies which levers move the probability needle. Value deltas, award size, and time horizon dominate the ranking, followed by procurement office and pricing signals.


In [33]:

def prettify_feature(raw_name: str) -> str:
    cleaned = raw_name.split('__', 1)[-1] if '__' in raw_name else raw_name
    return cleaned.replace('_', ' ').title()

feature_importances = artifacts.feature_importances.copy()
feature_importances['friendly'] = feature_importances['feature'].apply(prettify_feature)

top_features = feature_importances.head(15)
fig = go.Figure(
    data=[
        go.Bar(
            x=top_features['importance'][::-1],
            y=top_features['friendly'][::-1],
            orientation='h',
            marker_color='#636efa',
        )
    ]
)
fig.update_layout(
    title='Top feature contributions (permutation importance)',
    template='plotly_white',
    height=520,
    xaxis_title='Importance (mean performance drop)',
)
iplot(fig)

top_features[['friendly', 'importance']]


,friendly,importance
0,Transaction Description,0.03
1,Parent Award Agency Name,0.02
2,Total Dollars Obligated,0.01
3,Obligation Per Day,0.01
4,Awarding Office Name,0.01
5,Base And All Options Value,0.01
6,Receives Financial Assistance,0.01
7,National Interest Action,0.01
8,Awarding Agency Name,0.01
9,Solicitation To Award Days,0.01


## How top predictors influence risk

Management often asks *why* the model makes confident calls. The plots below show how the predicted probability shifts across the most influential quantitative levers.

In [45]:
# Diagnostic: Check dtypes of boolean features in dataset
print("Checking dtypes of boolean features in dataset:")
bool_cols_in_dataset = [col for col in BOOLEAN_FEATURES if col in dataset.columns]
print(f"\nFound {len(bool_cols_in_dataset)} boolean features in dataset")
print("\nSample of dtypes:")
for col in bool_cols_in_dataset[:10]:
    print(f"  {col}: {dataset[col].dtype}")

# Check if any are actually bool dtype
bool_dtype_cols = [col for col in bool_cols_in_dataset if dataset[col].dtype == 'bool']
print(f"\n⚠️ Columns with bool dtype: {len(bool_dtype_cols)}")
if bool_dtype_cols:
    print("First 5:", bool_dtype_cols[:5])

Checking dtypes of boolean features in dataset:

Found 86 boolean features in dataset

Sample of dtypes:
  small_disadvantaged_business: bool
  veteran_owned_business: bool
  service_disabled_veteran_owned_business: bool
  woman_owned_business: bool
  women_owned_small_business: bool
  economically_disadvantaged_women_owned_small_business: bool
  joint_venture_women_owned_small_business: bool
  joint_venture_economic_disadvantaged_women_owned_small_bus: bool
  historically_underutilized_business_zone_hubzone_firm: bool
  c8a_program_participant: bool

⚠️ Columns with bool dtype: 86
First 5: ['small_disadvantaged_business', 'veteran_owned_business', 'service_disabled_veteran_owned_business', 'woman_owned_business', 'women_owned_small_business']


In [51]:

scored_dataset = dataset.copy()
# Ensure boolean columns remain as int after copy
for bool_col in BOOLEAN_FEATURES:
    if bool_col in scored_dataset.columns:
        scored_dataset[bool_col] = scored_dataset[bool_col].astype(int)

scored_dataset['predicted_risk'] = artifacts.pipeline.predict_proba(scored_dataset[feature_columns])[:, 1]

feature_importances = artifacts.feature_importances.copy()

def raw_feature_name(prefixed_name: str) -> str:
    return prefixed_name.split('__', 1)[-1]

numeric_features_ranked = []
for feature_name in feature_importances['feature']:
    if feature_name.startswith('num__'):
        raw_name = raw_feature_name(feature_name)
        if raw_name in scored_dataset.columns and raw_name not in numeric_features_ranked:
            numeric_features_ranked.append(raw_name)
    if len(numeric_features_ranked) == 3:
        break

impact_fig = make_subplots(rows=1, cols=len(numeric_features_ranked), subplot_titles=[prettify_feature(name) for name in numeric_features_ranked])

for position, feature_name in enumerate(numeric_features_ranked, start=1):
    working = scored_dataset[[feature_name, 'predicted_risk']].dropna()
    if working.empty:
        continue
    working['bin'] = pd.qcut(working[feature_name], q=6, duplicates='drop')
    summary = (
        working
        .groupby('bin')
        .agg(
            avg_level=(feature_name, 'median'),
            risk=('predicted_risk', 'mean'),
            sample_count=('predicted_risk', 'size')
        )
        .reset_index(drop=True)
    )
    impact_fig.add_trace(
        go.Scatter(
            x=summary['avg_level'],
            y=summary['risk'],
            mode='lines+markers',
            name=prettify_feature(feature_name),
            marker=dict(size=8),
            hovertemplate='Median level: %{x:,.2f}<br>Predicted risk: %{y:.2%}<br>Contracts: %{text}',
            text=summary['sample_count']
        ),
        row=1,
        col=position,
    )
    impact_fig.update_xaxes(title=prettify_feature(feature_name), row=1, col=position)
impact_fig.update_yaxes(title='Predicted modification probability', row=1, col=1)
impact_fig.update_layout(
    title='Probability response across top quantitative drivers',
    template='plotly_white',
    height=420,
    showlegend=False
)
iplot(impact_fig)

cat_features_ranked = []
for feature_name in feature_importances['feature']:
    if feature_name.startswith('cat__'):
        raw_name = raw_feature_name(feature_name)
        if raw_name in scored_dataset.columns and raw_name not in cat_features_ranked:
            cat_features_ranked.append(raw_name)
    if len(cat_features_ranked) == 1:
        break

if cat_features_ranked:
    cat_feature = cat_features_ranked[0]
    category_summary = (
        scored_dataset
        .groupby(cat_feature)
        .agg(
            contracts=('contract_award_unique_key', 'nunique'),
            avg_risk=('predicted_risk', 'mean')
        )
        .query('contracts >= 100')
        .sort_values('avg_risk', ascending=False)
        .head(8)
        .reset_index()
    )
    cat_fig = go.Figure(
        data=[
            go.Bar(
                x=category_summary[cat_feature],
                y=category_summary['avg_risk'],
                text=[f"{val:.1%}" for val in category_summary['avg_risk']],
                textposition='outside',
                marker_color='#ff7f0e'
            )
        ]
    )
    cat_fig.update_layout(
        title=f"Average predicted risk by {prettify_feature(cat_feature)} (top segments)",
        xaxis_title=prettify_feature(cat_feature),
        yaxis_title='Predicted modification probability',
        template='plotly_white',
        height=420
    )
    cat_fig.update_yaxes(range=[0, min(1, category_summary['avg_risk'].max() * 1.1)])
    iplot(cat_fig)


/tmp/ipykernel_17928/2927339327.py:32: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/tmp/ipykernel_17928/2927339327.py:32: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/tmp/ipykernel_17928/2927339327.py:32: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.




## Scoring the full portfolio for proactive account planning

The trained model scores every base award so we can contrast realized modification rates with predicted probabilities across solicitation approaches. The view filters to procedures with at least 100 security contracts to focus on scalable levers.


In [52]:

dataset_scored = dataset.copy()
# Ensure boolean columns remain as int after copy
for bool_col in BOOLEAN_FEATURES:
    if bool_col in dataset_scored.columns:
        dataset_scored[bool_col] = dataset_scored[bool_col].astype(int)

dataset_scored['predicted_risk'] = artifacts.pipeline.predict_proba(dataset_scored[feature_columns])[:, 1]

solicitation_view = (
    dataset_scored
    .groupby('solicitation_procedures')
    .agg(
        awards=('contract_award_unique_key', 'nunique'),
        actual_mod_rate=(TARGET_COLUMN, 'mean'),
        avg_predicted_risk=('predicted_risk', 'mean'),
        median_base_value=('base_and_all_options_value', 'median'),
    )
    .query('awards >= 100')
    .sort_values('avg_predicted_risk', ascending=False)
    .head(10)
    .reset_index()
)
solicitation_view['actual_pct'] = solicitation_view['actual_mod_rate'] * 100
solicitation_view['predicted_pct'] = solicitation_view['avg_predicted_risk'] * 100

display(solicitation_view[['solicitation_procedures', 'awards', 'actual_pct', 'predicted_pct', 'median_base_value']])

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=solicitation_view['solicitation_procedures'],
        y=solicitation_view['actual_pct'],
        name='Observed modification rate (%)',
        marker_color='#bcbd22',
    )
)
fig.add_trace(
    go.Scatter(
        x=solicitation_view['solicitation_procedures'],
        y=solicitation_view['predicted_pct'],
        name='Predicted modification probability (%)',
        mode='lines+markers',
        marker=dict(color='#d62728'),
    )
)
fig.update_layout(
    title='Observed vs predicted risk by solicitation procedure',
    template='plotly_white',
    height=540,
    xaxis_title='Solicitation procedure',
    yaxis_title='Percentage of contracts modified',
)
fig.update_xaxes(tickangle=35)
iplot(fig)


,solicitation_procedures,awards,actual_pct,predicted_pct,median_base_value
0,NEGOTIATED PROPOSAL/QUOTE,11972,89.29,89.36,"382,311.64"
1,SUBJECT TO MULTIPLE AWARD FAIR OPPORTUNITY,4517,83.29,83.37,"195,156.00"
2,SIMPLIFIED ACQUISITION,25824,79.00,78.79,"24,000.00"
3,ONLY ONE SOURCE,3872,70.53,71.38,"16,652.16"
4,,467,60.39,62.71,"5,872.00"



## Executive takeaways

- **Modification prevalence remains high (81%)**, confirming that security vendors must budget for scope drift even when initial statements of work appear narrow.
- **Award size and latent option value gaps are the strongest predictors**: contracts whose `base_and_all_options_value` materially exceeds the exercised baseline are far more likely to evolve, signalling clear upsell potential.
- **Agency and office effects matter**: a handful of DHS and DOJ components exceed 90% modification rates, while others stay below 60%, suggesting differentiated governance maturity and the need for tailored change-control playbooks.
- **Solicitation levers remain actionable**: negotiated procurement methods and performance-based service acquisitions cluster at the top of the predicted risk ranking, meaning vendors can push for change-order protections whenever those procedures surface.
- **Model accuracy (ROC-AUC ≈ 0.94) enables reliable early-warning scores** that can feed pipeline reviews, empowering finance teams to stage working capital before amendments land.


Key Insights

Modification risk is structurally high: 46,771 base awards analyzed and 81% already experience downstream changes, underscoring the need to price contingencies even on “steady-state” security guard contracts.
Agency exposure is uneven: DHS alone shows a 95% modification rate across ~8.8k awards (≈$159B in base-and-options value), DOJ runs at 82% on 29k awards, while Interior drops to ~73%; capture teams should therefore tailor change-control strategies by agency.
Deal size and runway drive instability: contracts above ~$2.1M (top decile) and with ~1-year planned durations face ~93% modification probability, confirming that latent option value and longer schedules amplify scope drift.
Predictive model (HistGradientBoosting, 80/20 split) reaches accuracy 0.89, recall 0.97, ROC-AUC 0.94, AP 0.99—sufficient for early-warning dashboards that flag at-risk awards before execution.
Feature drivers align with strategic intuition: the gap between current vs. base value, total option headroom, planned duration, and the awarding office/agency dominate permutation importance, indicating that both value engineering and buyer behavior explain change frequency.
Solicitation levers remain actionable: negotiated proposals and multiple-award fair opportunities run observed/predicted modification rates near 89% and 83% respectively, suggesting vendors should insist on formal change-order protections whenever those procedures appear; simplified acquisitions still sit near 79%, so even “small and fast” buys aren’t immune.

## Export visualizations for documentation

The following cell saves all key visualizations to the `docs/ML/images/` folder for use in executive reports and presentations.

In [53]:
import plotly.io as pio
from pathlib import Path

# Create output directory
output_dir = PROJECT_ROOT / 'docs' / 'ML' / 'images'
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Saving visualizations to: {output_dir}")

# Helper function for value segment labels
def segment_label(interval):
    if pd.isna(interval):
        return 'Unknown'
    left = interval.left / 1e6
    right = interval.right / 1e6
    return f"${left:,.1f}M – ${right:,.1f}M"

# Helper function for feature names
def prettify_feature(raw_name: str) -> str:
    cleaned = raw_name.split('__', 1)[-1] if '__' in raw_name else raw_name
    return cleaned.replace('_', ' ').title()

# Re-generate and save all key visualizations

# 1. Agency modification rates chart
top_agencies_viz = (
    dataset
    .groupby('awarding_agency_name')
    .agg(
        awards=('contract_award_unique_key', 'nunique'),
        modification_rate=(TARGET_COLUMN, 'mean'),
        total_base_value=('base_and_all_options_value', 'sum'),
    )
    .sort_values('modification_rate', ascending=False)
    .head(12)
    .reset_index()
)
top_agencies_viz['mod_rate_pct'] = top_agencies_viz['modification_rate'] * 100
top_agencies_viz['total_base_value_m'] = top_agencies_viz['total_base_value'] / 1e6

fig1 = make_subplots(specs=[[{"secondary_y": True}]])
fig1.add_trace(
    go.Bar(
        x=top_agencies_viz['mod_rate_pct'],
        y=top_agencies_viz['awarding_agency_name'],
        orientation='h',
        name='Contracts modified (%)',
        marker_color='#1f77b4',
        hovertemplate='%{y}<br>Modification rate: %{x:.1f}%<extra></extra>',
    ),
    secondary_y=False,
)
fig1.add_trace(
    go.Scatter(
        x=top_agencies_viz['total_base_value_m'],
        y=top_agencies_viz['awarding_agency_name'],
        mode='lines+markers',
        name='Base value (USD millions)',
        marker=dict(color='#ff7f0e', size=8),
        hovertemplate='%{y}<br>Base value: %{x:.1f}M<extra></extra>',
    ),
    secondary_y=True,
)
fig1.update_layout(
    title='Agencies with the highest modification intensity',
    template='plotly_white',
    height=520,
    legend=dict(orientation='h', x=0, y=1.1),
    font=dict(size=12),
)
fig1.update_yaxes(categoryorder='total ascending')
fig1.update_xaxes(title='Modification rate (%)')
fig1.update_yaxes(title='Agency', secondary_y=False)
fig1.update_yaxes(title='Base value (USD millions)', secondary_y=True)

pio.write_image(fig1, output_dir / '01_agency_modification_rates.png', width=1200, height=600, scale=2)
print("✓ Saved: 01_agency_modification_rates.png")

# 2. Contract value vs modification risk
value_ready_viz = dataset.dropna(subset=['base_and_all_options_value']).copy()
value_ready_viz = value_ready_viz[value_ready_viz['base_and_all_options_value'] > 0]
value_ready_viz['value_segment'] = pd.qcut(value_ready_viz['base_and_all_options_value'], 10, duplicates='drop')

value_summary_viz = (
    value_ready_viz
    .groupby('value_segment')
    .agg(
        awards=('contract_award_unique_key', 'nunique'),
        modification_rate=(TARGET_COLUMN, 'mean'),
        median_duration=('planned_duration_days', 'median'),
    )
    .reset_index()
)

value_summary_viz['segment_label'] = value_summary_viz['value_segment'].apply(segment_label)
value_summary_viz['mod_rate_pct'] = value_summary_viz['modification_rate'] * 100

fig2 = make_subplots(specs=[[{"secondary_y": True}]])
fig2.add_trace(
    go.Bar(
        x=value_summary_viz['segment_label'],
        y=value_summary_viz['mod_rate_pct'],
        name='Contracts modified (%)',
        marker_color='#17becf',
    ),
    secondary_y=False,
)
fig2.add_trace(
    go.Scatter(
        x=value_summary_viz['segment_label'],
        y=value_summary_viz['median_duration'],
        name='Median planned duration (days)',
        mode='lines+markers',
        marker=dict(color='#7f7f7f'),
    ),
    secondary_y=True,
)
fig2.update_layout(
    title='Modification exposure by contract value decile',
    template='plotly_white',
    height=540,
    xaxis_title='Base and all options value decile',
    font=dict(size=12),
)
fig2.update_yaxes(title='Modification rate (%)', secondary_y=False)
fig2.update_yaxes(title='Median duration (days)', secondary_y=True)

pio.write_image(fig2, output_dir / '02_value_decile_modification_risk.png', width=1200, height=600, scale=2)
print("✓ Saved: 02_value_decile_modification_risk.png")

# 3. ROC Curve
fpr_viz, tpr_viz, _ = roc_curve(artifacts.y_test, artifacts.y_pred_proba)

fig3 = go.Figure()
fig3.add_trace(
    go.Scatter(
        x=fpr_viz,
        y=tpr_viz,
        mode='lines',
        name='Model ROC',
        line=dict(color='#1f77b4', width=3),
        hovertemplate='False-positive rate: %{x:.3f}<br>True-positive rate: %{y:.3f}<extra></extra>'
    )
)
fig3.add_trace(
    go.Scatter(
        x=[0, 1],
        y=[0, 1],
        mode='lines',
        name='Chance line',
        line=dict(color='#aaaaaa', dash='dash')
    )
)
fig3.update_layout(
    title='ROC curve — probability of catching future modifications',
    xaxis_title='False-positive rate',
    yaxis_title='True-positive rate',
    template='plotly_white',
    height=500,
    font=dict(size=12),
)

pio.write_image(fig3, output_dir / '03_roc_curve.png', width=900, height=600, scale=2)
print("✓ Saved: 03_roc_curve.png")

# 4. Precision-Recall Curve
precision_viz, recall_viz, _ = artifacts.precision_recall_curve

fig4 = go.Figure()
fig4.add_trace(
    go.Scatter(
        x=recall_viz,
        y=precision_viz,
        mode='lines',
        name='Precision-Recall',
        line=dict(color='#d62728', width=3),
        hovertemplate='Recall: %{x:.3f}<br>Precision: %{y:.3f}<extra></extra>'
    )
)
fig4.update_layout(
    title='Precision–recall curve — focus on reliable warnings',
    xaxis_title='Recall (share of modified awards captured)',
    yaxis_title='Precision (share of flagged awards that truly modify)',
    template='plotly_white',
    height=500,
    yaxis=dict(range=[0, 1.05]),
    font=dict(size=12),
)

pio.write_image(fig4, output_dir / '04_precision_recall_curve.png', width=900, height=600, scale=2)
print("✓ Saved: 04_precision_recall_curve.png")

# 5. Confusion Matrix
cmatrix_viz = artifacts.confusion_matrix
cm_text_viz = [[f"{val:,}" for val in row] for row in cmatrix_viz]

fig5 = go.Figure(
    data=go.Heatmap(
        z=cmatrix_viz,
        x=['Predicted no change', 'Predicted modification'],
        y=['Actual no change', 'Actual modification'],
        colorscale='Blues',
        showscale=False,
        text=cm_text_viz,
        texttemplate='%{text}',
        textfont=dict(size=16)
    )
)
fig5.update_layout(
    title='Confusion matrix — how the model performs on the hold-out test set',
    xaxis_title='Predicted class',
    yaxis_title='Actual class',
    template='plotly_white',
    height=500,
    font=dict(size=12),
)

pio.write_image(fig5, output_dir / '05_confusion_matrix.png', width=800, height=600, scale=2)
print("✓ Saved: 05_confusion_matrix.png")

# 6. Feature Importance
feature_importances_viz = artifacts.feature_importances.copy()
feature_importances_viz['friendly'] = feature_importances_viz['feature'].apply(prettify_feature)
top_features_viz = feature_importances_viz.head(15)

fig6 = go.Figure(
    data=[
        go.Bar(
            x=top_features_viz['importance'][::-1],
            y=top_features_viz['friendly'][::-1],
            orientation='h',
            marker_color='#636efa',
        )
    ]
)
fig6.update_layout(
    title='Top feature contributions (permutation importance)',
    template='plotly_white',
    height=600,
    xaxis_title='Importance (mean performance drop)',
    font=dict(size=12),
)

pio.write_image(fig6, output_dir / '06_feature_importance.png', width=1000, height=700, scale=2)
print("✓ Saved: 06_feature_importance.png")

# 7. Solicitation procedure comparison
dataset_viz = dataset.copy()
# Ensure boolean columns remain as int after copy
for bool_col in BOOLEAN_FEATURES:
    if bool_col in dataset_viz.columns:
        dataset_viz[bool_col] = dataset_viz[bool_col].astype(int)

solicitation_view_viz = (
    dataset_viz
    .assign(predicted_risk=artifacts.pipeline.predict_proba(dataset_viz[feature_columns])[:, 1])
    .groupby('solicitation_procedures')
    .agg(
        awards=('contract_award_unique_key', 'nunique'),
        actual_mod_rate=(TARGET_COLUMN, 'mean'),
        avg_predicted_risk=('predicted_risk', 'mean'),
        median_base_value=('base_and_all_options_value', 'median'),
    )
    .query('awards >= 100')
    .sort_values('avg_predicted_risk', ascending=False)
    .head(10)
    .reset_index()
)
solicitation_view_viz['actual_pct'] = solicitation_view_viz['actual_mod_rate'] * 100
solicitation_view_viz['predicted_pct'] = solicitation_view_viz['avg_predicted_risk'] * 100

fig7 = go.Figure()
fig7.add_trace(
    go.Bar(
        x=solicitation_view_viz['solicitation_procedures'],
        y=solicitation_view_viz['actual_pct'],
        name='Observed modification rate (%)',
        marker_color='#bcbd22',
    )
)
fig7.add_trace(
    go.Scatter(
        x=solicitation_view_viz['solicitation_procedures'],
        y=solicitation_view_viz['predicted_pct'],
        name='Predicted modification probability (%)',
        mode='lines+markers',
        marker=dict(color='#d62728'),
    )
)
fig7.update_layout(
    title='Observed vs predicted risk by solicitation procedure',
    template='plotly_white',
    height=600,
    xaxis_title='Solicitation procedure',
    yaxis_title='Percentage of contracts modified',
    font=dict(size=12),
)
fig7.update_xaxes(tickangle=35)

pio.write_image(fig7, output_dir / '07_solicitation_procedure_risk.png', width=1200, height=700, scale=2)
print("✓ Saved: 07_solicitation_procedure_risk.png")

print(f"\n✅ All visualizations saved successfully to {output_dir}")

Saving visualizations to: /mnt/ssd2/SAM_gov/docs/ML/images
✓ Saved: 01_agency_modification_rates.png
✓ Saved: 01_agency_modification_rates.png


/tmp/ipykernel_17928/2502272261.py:86: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



✓ Saved: 02_value_decile_modification_risk.png
✓ Saved: 03_roc_curve.png
✓ Saved: 04_precision_recall_curve.png
✓ Saved: 05_confusion_matrix.png
✓ Saved: 06_feature_importance.png
✓ Saved: 07_solicitation_procedure_risk.png

✅ All visualizations saved successfully to /mnt/ssd2/SAM_gov/docs/ML/images
